In [1]:
!nvidia-smi

Thu Jan 30 12:11:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 6000 Ada Gene...    Off |   00000000:01:00.0 Off |                    0 |
| 30%   28C    P8             14W /  300W |       2MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
import numpy as np
import cupy as cp
import pandas as pd_cpu
pd_cpu

<module 'pandas' from '/home/will/git/uvm-demo/venv/lib/python3.12/site-packages/pandas/__init__.py'>

In [6]:
def get_mem_tracker(curr_mem_df, phase, pd_var):
    
    # get sys stats
    host_mem_list = !free -m | awk 'NR==2{print $2, $3, $4}'
    gpu_mem_list = !nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv,noheader,nounits

    # create new mem df
    data=[['host', phase] + [mb // 1024 for mb in list(map(int, host_mem_list[0].split()))]]
    data += [['device', phase] + [mb // 1000 for mb in list(map(int, gpu_mem_list[0].split(', ')))]]    
    new_mem_df = pd_var.DataFrame(data=data, columns=['arch', 'phase', 'ttl', 'used', 'free'])    

    # concat 
    if curr_mem_df is not None and not curr_mem_df.empty:
        new_mem_df = pd_var.concat([curr_mem_df, new_mem_df])
    
    return new_mem_df

In [7]:
mem_df = get_mem_tracker(None, 'init', pd_var=pd_cpu)
mem_df

,arch,phase,ttl,used,free
0,host,init,62,1,51
1,device,init,46,0,45


### GB in bytes    

In [8]:
GB_bytes = 1024 ** 3

GB_bytes

1073741824

### float64 bytes

In [9]:
float64_bytes = np.float64().itemsize

float64_bytes

8

### GB / float64_bytes

In [10]:
float_array_1GB_len = GB_bytes // float64_bytes

float_array_1GB_len

134217728

### 1 GB array

In [11]:
ary_1GB = np.random.rand(float_array_1GB_len)

del GB_bytes, float64_bytes

ary_1GB.nbytes

1073741824

In [12]:
df_cpu_1GB = pd_cpu.DataFrame(data=ary_1GB)
del ary_1GB

### 40 GB array

In [13]:
del df_cpu_1GB

In [14]:
%%time
## 19 s

ary_40GB = np.random.rand(float_array_1GB_len * 40)

CPU times: user 14 s, sys: 3.44 s, total: 17.5 s
Wall time: 17.5 s


---

---

In [15]:
mem_df = get_mem_tracker(mem_df, phase='cpu_1GB', pd_var=pd_cpu)
mem_df

,arch,phase,ttl,used,free
0,host,init,62,1,51
1,device,init,46,0,45
0,host,cpu_1GB,62,41,11
1,device,cpu_1GB,46,0,45


In [16]:
%whos

Variable              Type         Data/Info
--------------------------------------------
ary_40GB              ndarray      5368709120: 5368709120 elems, type `float64`, 42949672960 bytes (40960.0 Mb)
cp                    module       <module 'cupy' from '/hom<...>ckages/cupy/__init__.py'>
float_array_1GB_len   int          134217728
get_mem_tracker       function     <function get_mem_tracker at 0x7a1ef0d5ca40>
mem_df                DataFrame         arch    phase  ttl  <...> cpu_1GB   46     0    45
np                    module       <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
pd_cpu                module       <module 'pandas' from '/h<...>ages/pandas/__init__.py'>


In [17]:
del ary_40GB

In [18]:
mem_df = get_mem_tracker(mem_df, phase='del_cpu_1GB', pd_var=pd_cpu)
mem_df

,arch,phase,ttl,used,free
0,host,init,62,1,51
1,device,init,46,0,45
0,host,cpu_1GB,62,41,11
1,device,cpu_1GB,46,0,45
0,host,del_cpu_1GB,62,1,51
1,device,del_cpu_1GB,46,0,45


In [19]:
%whos

Variable              Type         Data/Info
--------------------------------------------
cp                    module       <module 'cupy' from '/hom<...>ckages/cupy/__init__.py'>
float_array_1GB_len   int          134217728
get_mem_tracker       function     <function get_mem_tracker at 0x7a1ef0d5ca40>
mem_df                DataFrame         arch        phase  t<...>_cpu_1GB   46     0    45
np                    module       <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
pd_cpu                module       <module 'pandas' from '/h<...>ages/pandas/__init__.py'>


In [20]:
ary_40GB = cp.random.rand(float_array_1GB_len * 40)